# Superswaps

In [ ]:
#| default_exp superswap

In [ ]:
#| export

import json, requests
from sugar.swap import build_super_swap_data, SuperSwapData, SuperSwapQuote, setup_planner, SuperSwapDataInput
from sugar.token import Token
from sugar.helpers import get_salt, serialize_ica_calls
from sugar.config import hyperlane_relay_url, hyperlane_relayers
from sugar.chains import get_async_chain_from_token, AsyncChain
from typing import List, Dict, Any, Optional
from abc import ABC, abstractmethod

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export

class SuperswapRelayer(ABC):
    @abstractmethod
    def share_calls(self, calls: List[dict], salt: str, commitment_dispatch_tx: str, origin_domain: int) -> None:
        """Share calls with the relayer."""
        pass


class HTTPSuperswapRelayer(SuperswapRelayer):
    """HTTP-based relayer implementation."""
    def share_calls(self, calls: List[dict], salt: str, commitment_dispatch_tx: str, origin_domain: int) -> None:
        """
        Share calls with private relayer.
        
        Args:
            calls: List of call data dictionaries
            salt: Hex string salt value
            commitment_dispatch_tx: Transaction hash string
            origin_domain: Domain number
        """
        body = json.dumps({
            'commitmentDispatchTx': commitment_dispatch_tx,
            'originDomain': origin_domain,
            'calls': calls,
            'salt': salt,
            'relayers': hyperlane_relayers
        })
        print("body:")
        print(body)
        resp = requests.post(hyperlane_relay_url, headers={'Content-Type': 'application/json'}, data=body)
        print(f"Response status code: {resp.status_code}: {resp.text}")

        if not resp.ok:
            # Read response text
            response_text = resp.text
            error_msg = f"Failed to share calls with relayer: {resp.status_code} {response_text}"
            # Replace with your preferred error handling (toast equivalent)
            print(f"Error: {error_msg}")
            raise Exception(error_msg)
                

In [ ]:
#| export


# You'll need to import or define these constants
# from constants import HL_RELAY_URL

# TODO: add helper to inspect tx using https://explorer.hyperlane.xyz/?search

class AsyncSuperswap:
    def __init__(self, relayer: Optional[SuperswapRelayer] = None, chain_for_writes: Optional[AsyncChain] = None):
        self.chain_for_writes = chain_for_writes
        self.relayer = relayer or HTTPSuperswapRelayer()

    async def swap(self, from_token: Token, to_token: Token, amount: float, slippage: Optional[float] = None) -> str:
        quote = await self.get_super_quote(from_token=from_token, to_token=to_token, amount_in=25)
        print(f"SuperSwap quote: {quote}")
        return await self.swap_from_quote(quote=quote, slippage=0.1)

    async def get_super_quote(self, from_token: Token, to_token: Token, amount_in: float) -> SuperSwapQuote:
        from_chain, to_chain = get_async_chain_from_token(from_token), get_async_chain_from_token(to_token)
        async with from_chain, to_chain:
            origin_bridge_token, destination_bridge_token = await from_chain.get_superswap_connector_token(), await to_chain.get_superswap_connector_token()

        # hard code op -> uni swap for now
        starts_with_bridge_token = from_token.token_address == origin_bridge_token.token_address
        ends_with_bridge_token = to_token.token_address == destination_bridge_token.token_address
        origin_quote, destination_quote = None, None

        # we only need origin quote if we don't start with oUSDT
        if not starts_with_bridge_token:
            async with from_chain:
                origin_quote = await from_chain.get_quote(from_token, origin_bridge_token, amount_in)
                assert origin_quote is not None, "No origin quote found"

        # we need destination quote if we don't end with oUSDT
        if not ends_with_bridge_token:
            # XX: get_quote expects amount_in in "normal" (not wei) format, adjust accordingly
            bridged_amount = float(origin_quote.amount_out / 10 ** origin_bridge_token.decimals) if not starts_with_bridge_token else amount_in
            async with to_chain:
                destination_quote = await to_chain.get_quote(destination_bridge_token, to_token, bridged_amount)

        return SuperSwapQuote(
            from_token=from_token,
            to_token=to_token,
            from_bridge_token=origin_bridge_token,
            to_bridge_token=destination_bridge_token,
            amount_in=amount_in,
            origin_quote=origin_quote,
            destination_quote=destination_quote
        )

    async def swap_from_quote(self, quote: SuperSwapQuote, slippage: float, salt: Optional[str] = None):
        # TODO: remove this
        from_chain, to_chain = get_async_chain_from_token(quote.from_token), get_async_chain_from_token(quote.to_token)

        # assert from_chain.name == "OP" and to_chain .name == "Uni", "SuperSwap only supports OP -> Uni for now"

        origin_quote, bridged_amount, destination_quote = quote.origin_quote, quote.origin_quote.amount_out, quote.destination_quote

        async with from_chain, to_chain:
            if not from_chain.account: raise ValueError("Cannot superswap without an account. Please connect your wallet first.")
            origin_domain = await from_chain.get_domain(int(from_chain.settings.chain_id))
            # TODO: use to_chain.get_domain() when all chains support domains
            destination_domain = await from_chain.get_domain(int(to_chain.settings.chain_id))
            user_ica_address = await from_chain.get_remote_interchain_account(destination_domain)
            bridge_fee = await from_chain.get_bridge_fee(int(to_chain.id))
            xchain_fee = await from_chain.get_xchain_fee(destination_domain)
            total_fee = bridge_fee + xchain_fee if quote.to_token.token_address != quote.to_bridge_token.token_address else bridge_fee 
            
            print(f"bridge_fee: {bridge_fee}, xchain_fee: {xchain_fee}, total_fee: {total_fee}")

            swap_data = build_super_swap_data(SuperSwapDataInput(
                from_token=quote.from_token,
                to_token=quote.to_token,
                from_bridge_token=quote.from_bridge_token,
                to_bridge_token=quote.to_bridge_token,
                account=from_chain.account.address,
                user_ICA=user_ica_address,
                user_ICA_balance=await to_chain.get_user_ica_balance(user_ica_address),
                origin_domain=origin_domain,
                origin_bridge=from_chain.settings.bridge_contract_addr,
                origin_hook=await from_chain.get_ica_hook(),
                origin_ICA_router=from_chain.settings.interchain_router_contract_addr,
                destination_ICA_router=to_chain.settings.interchain_router_contract_addr,
                destination_router=to_chain.settings.swapper_contract_addr,
                destination_domain=destination_domain,
                slippage=slippage,
                bridged_amount=bridged_amount,
                swapper_contract_addr=to_chain.settings.swapper_contract_addr,
                destination_quote=destination_quote,
                bridge_fee=bridge_fee,
                xchain_fee=xchain_fee,
                salt=salt if salt else get_salt()
            ))

            origin_planner = setup_planner(
                quote=origin_quote,
                slippage=slippage,
                # money goes to the universal router (aka swapper) for bridging
                account=from_chain.settings.swapper_contract_addr, 
                router_address=from_chain.settings.swapper_contract_addr
            ) if origin_quote else None

            print(f"Origin planner: {origin_planner}")
            print(f"Origin planner: {origin_planner.commands}")
            for i in origin_planner.inputs: print(f"origin planner input {i.hex()}")

            cmds, inputs = "", [] 

            if origin_planner:
                cmds += origin_planner.commands
                inputs.extend(origin_planner.inputs)
            if swap_data.destination_planner:
                cmds += swap_data.destination_planner.commands.replace("0x", "") if cmds != "" else swap_data.destination_planner.commands
                inputs.extend(swap_data.destination_planner.inputs)

            return await self.write(chain=self.chain_for_writes or from_chain, cmds=cmds, inputs=inputs, quote=quote, swap_data=swap_data, total_fee=total_fee)

    async def write(self, chain: AsyncChain, quote: SuperSwapQuote, swap_data: SuperSwapData, cmds: str, inputs: List[bytes], total_fee: int) -> str:
        # >>>>>>>>>>>>>>>>>>> WRITE stuff <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
        swapper_contract_addr, from_token =  chain.settings.swapper_contract_addr, quote.from_token
        value = quote.amount_in * (10 ** from_token.decimals)
        # TODO: extend this to proper native token support
        message_fee = value + total_fee if quote.from_token.wrapped_token_address else total_fee
        async with chain:
            await chain.set_token_allowance(from_token, swapper_contract_addr, value)
            tx = await chain.sign_and_send_tx(chain.swapper.functions.execute(*[cmds, inputs]), value=message_fee)
            print(f"tx : {tx}")
            if swap_data.needs_relay:
                self.relayer.share_calls(
                    calls=serialize_ica_calls(swap_data.calls),
                    salt=swap_data.salt,
                    commitment_dispatch_tx=f'0x{tx["transactionHash"].hex()}',
                    origin_domain=swap_data.origin_domain
                )
            return f'0x{tx["transactionHash"].hex()}'
        

## Tests

In [ ]:
from sugar.helpers import require_supersim
from fastcore.test import test_eq

require_supersim()

In [ ]:
class MockSuperswapRelayer(SuperswapRelayer):
    """Mock relayer implementation for testing."""
    
    def __init__(self):
        self.calls_history: List[Dict[str, Any]] = []
    
    def share_calls(self, calls: List[dict], salt: str, commitment_dispatch_tx: str, origin_domain: int) -> None:
        """Mock implementation that records calls for verification."""
        call_data = {
            'calls': calls,
            'salt': salt,
            'commitment_dispatch_tx': commitment_dispatch_tx,
            'origin_domain': origin_domain
        }
        self.calls_history.append(call_data)
        print(f"Mock relayer received call: {call_data}")
    
    def get_last_call(self) -> Optional[Dict[str, Any]]:
        """Get the most recent call data."""
        return self.calls_history[-1] if self.calls_history else None
    
    def get_call_count(self) -> int:
        """Get the total number of calls made."""
        return len(self.calls_history)
    


In [ ]:
from sugar import AsyncOPChainSimnet, AsyncOPChain, AsyncLiskChain

relayer=MockSuperswapRelayer()

async with AsyncOPChain() as op, AsyncLiskChain() as lisk, AsyncOPChainSimnet() as op_sim:
    from_token = await op.get_token("0x9560e827aF36c94D2Ac33a39bCE1Fe78631088Db")
    to_token = await lisk.get_token("0xac485391EB2d7D88253a7F1eF18C37f4242D1A24")
    tx = await AsyncSuperswap(chain_for_writes=op_sim, relayer=relayer).swap(from_token, to_token, amount=20)
    assert(tx.startswith("0x"))
    test_eq(relayer.get_call_count(), 1)
    last_call = relayer.get_last_call()
    test_eq(type(last_call["salt"]), str)
    test_eq(type(last_call["origin_domain"]), int)

SuperSwap quote: SuperSwapQuote(from_token=Token(chain_id='10', chain_name='OP', token_address='0x9560e827aF36c94D2Ac33a39bCE1Fe78631088Db', symbol='VELO', decimals=18, listed=True, wrapped_token_address=None), to_token=Token(chain_id='1135', chain_name='Lisk', token_address='0xac485391EB2d7D88253a7F1eF18C37f4242D1A24', symbol='LSK', decimals=18, listed=True, wrapped_token_address=None), from_bridge_token=Token(chain_id='10', chain_name='OP', token_address='0x1217BfE6c773EEC6cc4A38b5Dc45B92292B6E189', symbol='oUSDT', decimals=6, listed=True, wrapped_token_address=None), to_bridge_token=Token(chain_id='1135', chain_name='Lisk', token_address='0x1217BfE6c773EEC6cc4A38b5Dc45B92292B6E189', symbol='oUSDT', decimals=6, listed=True, wrapped_token_address=None), amount_in=25, origin_quote=Quote(input=QuoteInput(from_token=Token(chain_id='10', chain_name='OP', token_address='0x9560e827aF36c94D2Ac33a39bCE1Fe78631088Db', symbol='VELO', decimals=18, listed=True, wrapped_token_address=None), to_tok

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()